In [1]:
from net import policyNet
import torchinfo

In [2]:
pNet = policyNet()
torchinfo.summary(pNet, input_size=(512, 5, 5, 5))

Layer (type:depth-idx)                        Output Shape              Param #
policyNet                                     [512, 26]                 --
├─Sequential: 1-1                             [512, 64, 5, 5]           --
│    └─Conv2d: 2-1                            [512, 64, 5, 5]           2,944
│    └─ReLU: 2-2                              [512, 64, 5, 5]           --
├─BasicBlock: 1-2                             [512, 64, 5, 5]           --
│    └─Sequential: 2-3                        [512, 64, 5, 5]           --
│    │    └─BN_Conv2d: 3-1                    [512, 64, 5, 5]           36,992
│    │    └─BN_Conv2d: 3-2                    [512, 64, 5, 5]           36,992
│    └─Identity: 2-4                          [512, 64, 5, 5]           --
├─BasicBlock: 1-3                             [512, 64, 5, 5]           --
│    └─Sequential: 2-5                        [512, 64, 5, 5]           --
│    │    └─BN_Conv2d: 3-3                    [512, 64, 5, 5]           36,992
│    

In [3]:
import onnx
import torch
import torch.nn as nn
import onnxoptimizer

In [4]:
pNet = policyNet()
def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
pNet.apply(init_weights)
pNet.output.append(nn.Softmax(dim=1))

pNet.eval()
x = torch.randn(1,5,5,5)
torch.set_default_dtype(torch.float32)
torch.onnx.export(pNet, x, 'model.onnx', input_names=['gameBoard'],output_names=['policy'],
                dynamic_axes={'gameBoard':{0:"batch"},'policy':{0:"batch"}})

model = onnx.load('model.onnx') # load the onnx model
onnx.checker.check_model(model)         # check the onnx model
model = onnxoptimizer.optimize(model)
onnx.checker.check_model(model)
onnx.save(model, 'model.onnx')